In [1]:
import h2o
from h2o.automl import H2OAutoML 


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,6 mins 54 secs
H2O_cluster_timezone:,Europe/Moscow
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_Alkzir_2qov47
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,89.6 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv('../deaths.csv')
df

,Unnamed: 0,Country,Population,Area,Density,GDP,handwashing_facilities,hospital_beds_per_100k,Airports,Lat,...,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20
0,0,Afghanistan,38928341.0,652230.0,58.3257,74792.0,37.746,0.50,22.0,33.000000,...,193.0,205.0,216.0,218.0,219.0,220.0,227.0,235.0,246.0,249.0
1,1,Albania,2877800.0,28748.0,100.2128,38078.0,NaN,2.89,5.0,41.153300,...,31.0,31.0,31.0,32.0,32.0,33.0,33.0,33.0,33.0,33.0
2,2,Algeria,43851043.0,2381741.0,18.0763,638363.0,83.741,1.90,44.0,28.033900,...,575.0,582.0,592.0,600.0,609.0,617.0,623.0,630.0,638.0,646.0
3,3,Andorra,77265.0,468.0,164.8333,NaN,NaN,NaN,NaN,42.506300,...,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0
4,4,Angola,32866268.0,1246700.0,25.5276,199329.0,26.664,NaN,25.0,-11.202700,...,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,179,West Bank and Gaza,5101416.0,NaN,NaN,NaN,NaN,NaN,NaN,31.952200,...,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
184,180,Western Sahara,597330.0,266000.0,2.1897,NaN,NaN,NaN,3.0,24.215500,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
185,181,Yemen,29825968.0,527968.0,55.2343,70414.0,49.542,0.70,11.0,15.552727,...,33.0,33.0,39.0,42.0,44.0,49.0,53.0,57.0,65.0,77.0
186,182,Zambia,18383956.0,752612.0,23.7321,73662.0,13.938,2.00,13.0,-15.416700,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0


In [5]:
df['handwashing_facilities'] = df['handwashing_facilities'].fillna(df['handwashing_facilities'].mean())
df['hospital_beds_per_100k'] = df['hospital_beds_per_100k'].fillna(df['hospital_beds_per_100k'].median())

In [6]:
new_df = df.dropna()
new_df

,Unnamed: 0,Country,Population,Area,Density,GDP,handwashing_facilities,hospital_beds_per_100k,Airports,Lat,...,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20
0,0,Afghanistan,38928341.0,652230.0,58.3257,74792.0,37.7460,0.5000,22.0,33.000000,...,193.0,205.0,216.0,218.0,219.0,220.0,227.0,235.0,246.0,249.0
1,1,Albania,2877800.0,28748.0,100.2128,38078.0,50.4595,2.8900,5.0,41.153300,...,31.0,31.0,31.0,32.0,32.0,33.0,33.0,33.0,33.0,33.0
2,2,Algeria,43851043.0,2381741.0,18.0763,638363.0,83.7410,1.9000,44.0,28.033900,...,575.0,582.0,592.0,600.0,609.0,617.0,623.0,630.0,638.0,646.0
4,4,Angola,32866268.0,1246700.0,25.5276,199329.0,26.6640,2.3585,25.0,-11.202700,...,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
5,5,Antigua and Barbuda,97928.0,442.0,219.7240,2506.0,50.4595,3.8000,2.0,17.060800,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,176,Uzbekistan,33469199.0,447400.0,73.7186,310738.0,50.4595,4.0000,13.0,41.377500,...,13.0,13.0,13.0,13.0,13.0,14.0,14.0,14.0,14.0,14.0
182,178,Vietnam,97338583.0,331212.0,291.2398,1035051.0,85.8470,2.6000,25.0,16.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
185,181,Yemen,29825968.0,527968.0,55.2343,70414.0,49.5420,0.7000,11.0,15.552727,...,33.0,33.0,39.0,42.0,44.0,49.0,53.0,57.0,65.0,77.0
186,182,Zambia,18383956.0,752612.0,23.7321,73662.0,13.9380,2.0000,13.0,-15.416700,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0


In [7]:
data = df.T[11:]
data = data.astype('float64')

In [8]:
data.T

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,193.0,205.0,216.0,218.0,219.0,220.0,227.0,235.0,246.0,249.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31.0,31.0,31.0,32.0,32.0,33.0,33.0,33.0,33.0,33.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,575.0,582.0,592.0,600.0,609.0,617.0,623.0,630.0,638.0,646.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,33.0,33.0,39.0,42.0,44.0,49.0,53.0,57.0,65.0,77.0
186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0


In [ ]:
train = h2o.H2OFrame(np.array(data)[:, :105].T)
test  = h2o.H2OFrame(np.array(data)[:, 105:].T)

# Identify predictors and response
x = train.columns
y = "C130"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y]
test[y] = test[y]

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)

aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leaderboard
# lb = aml.leaderboard
# lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

lb = aml.leaderboard
lbdf = lb.as_data_frame()
lbdf.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |
20:52:16.422: AutoML: XGBoost is not available; skipping it.

████████████
20:52:29.573: Skipping training of model GBM_5_AutoML_20200611_205216 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20200611_205216.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 105.0.


██████████████████████████████████